In [3]:
from pyspark.sql import SparkSession

# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("covid") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local appName=covid>

In [ ]:
import zipfile
import os

# Ruta al archivo ZIP que deseas desempaquetar
archivo_zip = "covid.zip"

# Obtener la ruta de la carpeta donde se encuentra el archivo ZIP
carpeta_actual = os.path.dirname(archivo_zip)
print(carpeta_actual)
# Crear un objeto ZipFile
with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
    # Extraer todo el contenido en la carpeta actual
    zip_ref.extractall(carpeta_actual)

print("Archivo ZIP desempaquetado en la misma carpeta.")


In [4]:
import pandas as pd
import os
for filename in os.listdir(os.getcwd()):
    if '.csv' in filename :
        #print(filename)
        # Leer el archivo CSV en un DataFrame de Spark
        
        df = spark.read.option("header", "true").csv(filename)
        # Guardar el DataFrame en formato Parquet
        filename=filename.split('.')[0]
        parquet_output_path = f'tmp/'+filename+'/'
        df.repartition(2).write.mode("overwrite").parquet(parquet_output_path) 

In [14]:
def read_parquet_past_to_pandas(path):
      
            parqDF=spark.read.parquet('tmp/'+path+'/*.parquet')
            #parqDF.show()
            
            df=parqDF.pandas_api()
            return df
            
            #print(parqPandas_clean_complete.head())

In [22]:
parqPandas_country_wise_latest=read_parquet_past_to_pandas('country_wise_latest')
parqPandas_covid_19_clean_complete=read_parquet_past_to_pandas('covid_19_clean_complete')
parqPandas_day_wise=read_parquet_past_to_pandas('day_wise')
parqPandas_full_grouped=read_parquet_past_to_pandas('full_grouped')
parqPandas_usa_country_wise=read_parquet_past_to_pandas('usa_county_wise')
parqPandas_worldometer_data=read_parquet_past_to_pandas('worldometer_data')

In [23]:
parqPandas_country_wise_latest.head()
parqPandas_country_wise_latest.rename(columns={"Country/Region":"country_region","Confirmed":"confirmed","Deaths":"deaths","Recovered":"recovered",
                  "Active":"actived","New cases":"new_cases","New deaths":"new_deaths","New recovered":"new_recovered","Deaths / 100 Cases":"deaths100cases","Recovered / 100 Cases":"recoverded100cases","Deaths / 100 Recovered":"deaths100recovered","Confirmed last week":"confirmedlastweek","1 week change":"1weekchange","1 week % increase":"1weekperincrease","WHO Region":"who_region"},inplace=True)

In [54]:
#casteo country_wise_latest
parqPandas_country_wise_latest[['confirmed', 'deaths', 'recovered','actived','new_cases','new_deaths','new_recovered']] = parqPandas_country_wise_latest[['confirmed', 'deaths', 'recovered','actived','new_cases','new_deaths','new_recovered']].astype(int)
parqPandas_country_wise_latest[['confirmedlastweek','1weekchange','1weekperincrease']] = parqPandas_country_wise_latest[['confirmedlastweek','1weekchange','1weekperincrease']].astype(int)
parqPandas_country_wise_latest[['country_region','who_region']] = parqPandas_country_wise_latest[['country_region','who_region']].astype(str)
parqPandas_country_wise_latest[['deaths100cases','recoverded100cases','deaths100recovered']] = parqPandas_country_wise_latest[['deaths100cases','recoverded100cases','deaths100recovered']].astype(float)
parqPandas_country_wise_latest.head()
#df[['one', 'two', 'three']] = df[['one', 'two', 'three']].astype(str)
# add as many column names as you like.

,country_region,confirmed,deaths,recovered,actived,new_cases,new_deaths,new_recovered,deaths100cases,recoverded100cases,deaths100recovered,confirmedlastweek,1weekchange,1weekperincrease,who_region
0,Sudan,11424,720,5939,4765,39,3,49,6.30,51.99,12.12,10992,432,3,Eastern Mediterranean
1,Ethiopia,14547,228,6386,7933,579,5,170,1.57,43.90,3.57,10207,4340,42,Africa
2,Liechtenstein,86,1,81,4,0,0,0,1.16,94.19,1.23,86,0,0,Europe
3,Hungary,4448,596,3329,523,13,0,0,13.40,74.84,17.90,4339,109,2,Europe
4,Qatar,109597,165,106328,3104,292,0,304,0.15,97.02,0.16,107037,2560,2,Eastern Mediterranean


In [69]:
#casteo covid_19_clean_complete
parqPandas_covid_19_clean_complete.rename(columns={"Date":"date","Province/State":"province_state","Country/Region":"country_region","Lat":"lat",
                  "Long":"long","Confirmed":"confirmed","Deaths":"deaths","Recovered":"recovered","Active":"actived","WHO Region":"who_region"},inplace=True)
parqPandas_covid_19_clean_complete[['confirmed', 'deaths', 'recovered','actived']] = parqPandas_covid_19_clean_complete[['confirmed', 'deaths', 'recovered','actived']].astype(int)
parqPandas_covid_19_clean_complete[['lat', 'long']] = parqPandas_covid_19_clean_complete[['lat', 'long']].astype(float)
parqPandas_covid_19_clean_complete[['province_state', 'country_region', 'who_region']] = parqPandas_covid_19_clean_complete[['province_state', 'country_region']].astype(str)
#parqPandas_covid_19_clean_complete[['date']] = parqPandas_covid_19_clean_complete[['date']].astype(str)
#parqPandas_covid_19_clean_complete['date'] = pd.to_datetime(parqPandas_covid_19_clean_complete['date'])

#parqPandas_covid_19_clean_complete['date'] = pd.to_datetime(parqPandas_covid_19_clean_complete['date'])
#parqPandas_covid_19_clean_complete['date'] = pd.to_datetime(parqPandas_covid_19_clean_complete['date'], format='%Y-%m-%d')
parqPandas_covid_19_clean_complete['date'] = parqPandas_covid_19_clean_complete['date'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d'))

parqPandas_covid_19_clean_complete.head()


,province_state,country_region,lat,long,date,confirmed,deaths,recovered,actived,who_region
0,None,Japan,36.204824,138.252924,2020-06-07,17039,917,14990,1132,Western Pacific
1,Beijing,China,40.182400,116.414200,2020-04-04,585,8,438,139,Western Pacific
2,None,Zambia,-13.133897,27.849332,2020-06-27,1531,21,1233,277,Africa
3,Yukon,Canada,64.282300,-135.000000,2020-04-03,6,0,0,6,Americas
4,None,Liberia,6.428055,-9.429499,2020-03-21,3,0,0,3,Africa


In [84]:
#parqPandas_day_wise=parqPandas_day_wise.rename(columns={"Confirmed":"confirmed"})
parqPandas_day_wise.rename(columns={"Date":"date","Province/State":"state","Country/Region":"country","Lat":"lat",
                  "Long":"long","Confirmed":"confirmed","Deaths":"deaths","Recovered":"recovered","Active":"actived","WHO Region":"who_region","Deaths / 100 Cases":"deaths100cases","Recovered / 100 Cases":"recoverded100cases","Deaths / 100 Recovered":"deaths100recovered","New cases":"new_cases","New deaths":"new_deaths","New recovered":"new_recovered",'No. of countries':'numcountries'},inplace=True)

parqPandas_day_wise[['confirmed', 'deaths', 'recovered','actived','new_deaths','new_recovered','numcountries']] = parqPandas_day_wise[['confirmed', 'deaths', 'recovered','actived','new_deaths','new_recovered','numcountries']].astype(int)
parqPandas_day_wise[['deaths100cases', 'recoverded100cases','deaths100recovered']] = parqPandas_day_wise[['deaths100cases', 'recoverded100cases','deaths100recovered']].astype(float)
parqPandas_day_wise['date'] = parqPandas_day_wise['date'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d'))

parqPandas_day_wise.head()

,date,confirmed,deaths,recovered,actived,new_cases,new_deaths,new_recovered,deaths100cases,recoverded100cases,deaths100recovered,numcountries
0,2020-06-20,8805336,465222,4298603,4041511,157552,4249,115305,5.28,48.82,10.82,187
1,2020-04-28,3108149,221974,884680,2001495,75404,6463,32298,7.14,28.46,25.09,184
2,2020-02-17,72815,1868,11951,58996,1936,98,1632,2.57,16.41,15.63,27
3,2020-01-29,6166,133,125,5908,588,2,18,2.16,2.03,106.40,18
4,2020-01-31,9927,213,219,9495,1693,42,78,2.15,2.21,97.26,24


In [31]:
#parqPandas_day_wise=parqPandas_day_wise.rename(columns={"Confirmed":"confirmed"})
parqPandas_full_grouped.rename(columns={"Date":"date","Province/State":"state","Country/Region":"country","Confirmed":"confirmed","Deaths":"deaths","Recovered":"recovered","Active":"actived","WHO Region":"who_region","New cases":"new_cases","New deaths":"new_deaths","New recovered":"new_recovered"},inplace=True)

parqPandas_full_grouped[['confirmed', 'deaths', 'recovered','actived','new_deaths','new_recovered','numcountries','new_cases']] = parqPandas_full_grouped[['confirmed', 'deaths', 'recovered','actived','new_deaths','new_recovered','numcountries','new_cases']].astype(int)
#parqPandas_full_grouped[['deaths100cases', 'recoverded100cases','deaths100recovered']] = parqPandas_full_grouped[['deaths100cases', 'recoverded100cases','deaths100recovered']].astype(float)
parqPandas_full_grouped['date'] = parqPandas_full_grouped['date'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d'))

parqPandas_full_grouped.head()

,date,country,confirmed,deaths,recovered,actived,new_cases,new_deaths,new_recovered,who_region
0,2020-03-09,Turkey,0,0,0,0,0,0,0,Europe
1,2020-05-06,Jordan,473,9,377,87,2,0,2,Eastern Mediterranean
2,2020-01-27,Belarus,0,0,0,0,0,0,0,Europe
3,2020-02-20,Bangladesh,0,0,0,0,0,0,0,South-East Asia
4,2020-06-26,Czechia,11038,349,7668,3021,168,4,19,Europe


In [99]:
from datetime import datetime
from dateutil.parser import parse

# Función para convertir fechas
def convert_date(date_string):
    try:
        return parse(date_string).date()
    except ValueError:
        return None
parqPandas_usa_country_wise.rename(columns={"UID":"uid","Province/State":"province_state ","Country/Region":"country_region","Lat":"lat","Long_":"long","FIPS":"fips","Active":"actived","WHO Region":"who_region","New cases":"new_cases","New deaths":"new_deaths","Date":"date","Confirmed":"confirmed","Deaths":"deaths"},inplace=True)
parqPandas_usa_country_wise[['uid', 'code3', 'fips','confirmed','deaths']] = parqPandas_usa_country_wise[['uid', 'code3', 'fips','confirmed','deaths']].astype(int)
parqPandas_usa_country_wise[['lat', 'long']] = parqPandas_usa_country_wise[['lat', 'long']].astype(float)
parqPandas_usa_country_wise['date'] = parqPandas_usa_country_wise['date'].apply(convert_date)
parqPandas_usa_country_wise.head()


,uid,iso2,iso3,code3,fips,Admin2,Province_State,Country_Region,lat,long,date,confirmed,deaths
0,84030063,US,USA,840,30063,Missoula,Montana,US,47.037286,-113.921351,2020-05-28,39,1
1,84008107,US,USA,840,8107,Routt,Colorado,US,40.485965,-106.990555,2020-07-05,77,6
2,84055087,US,USA,840,55087,Outagamie,Wisconsin,US,44.415789,-88.465650,2020-06-16,345,8
3,84012005,US,USA,840,12005,Bay,Florida,US,30.265487,-85.621226,2020-06-11,135,4
4,84016035,US,USA,840,16035,Clearwater,Idaho,US,46.674599,-115.662239,2020-01-26,0,0


In [98]:
pip install python-dateutil


Note: you may need to restart the kernel to use updated packages.


In [101]:
parqPandas_worldometer_data.rename(columns={"Date":"date","Province/State":"state","Country/Region":"country","Tests/1M pop":"tests1Mpop",
                  "Serious,Critical":"serious_critical","Tot Cases/1M pop":"totcases1Mpop","Deaths/1M pop":"deaths1Mpop","Recovered":"recovered","Active":"actived","WHO Region":"who_region","Deaths / 100 Cases":"deaths100cases","Recovered / 100 Cases":"recoverded100cases","Deaths / 100 Recovered":"deaths100recovered","New cases":"new_cases","New deaths":"new_deaths","New recovered":"new_recovered",'No. of countries':'numcountries'},inplace=True)
parqPandas_worldometer_data[['Population', 'TotalCases', 'NewCases','TotalDeaths','NewDeaths','TotalRecovered','NewRecovered','ActiveCases','serious_critical']] = parqPandas_worldometer_data[['Population', 'TotalCases', 'NewCases','TotalDeaths','NewDeaths','TotalRecovered','NewRecovered','ActiveCases','serious_critical']].astype(int)
parqPandas_worldometer_data[['totcases1Mpop', 'deaths1Mpop', 'TotalTests','tests1Mpop']] = parqPandas_worldometer_data[['totcases1Mpop', 'deaths1Mpop', 'TotalTests','tests1Mpop']].astype(int)

parqPandas_worldometer_data.head()

,country,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,serious_critical,totcases1Mpop,deaths1Mpop,TotalTests,tests1Mpop,who_region
0,Greenland,North America,56780,14,NaN,NaN,NaN,14,NaN,0,NaN,247,NaN,5977,105266,Europe
1,Hungary,Europe,9657785,4597,NaN,600.0,NaN,3463,NaN,534,8.0,476,62.0,352546,36504,Europe
2,Israel,Asia,9197590,79559,NaN,576.0,NaN,53427,NaN,25556,358.0,8650,63.0,1872453,203581,Europe
3,Latvia,Europe,1883936,1275,NaN,32.0,NaN,1070,NaN,173,NaN,677,17.0,207909,110359,Europe
4,Saint Lucia,North America,183712,25,NaN,NaN,NaN,24,NaN,1,NaN,136,NaN,3895,21202,Americas
